# Sentiment Analysis with Multiclass SVM

In this case we are going to use scikit-learn and Gensim's Doc2Vec.

In [1]:
# CONFIGS
doc2vec_model_file = "word2vec/d2v-not-padded.model"

# Can easily swap with other datasets if you want
positive_review_txts = "aclImdb/train/pos/*.txt"
negative_review_txts = "aclImdb/train/neg/*.txt"
positive_review_vals = "aclImdb/test/pos/*.txt"
negative_review_vals = "aclImdb/test/neg/*.txt"

positive_y = 1
negative_y = 0

In [2]:
from sklearn import svm
from gensim.models import Doc2Vec
from bs4 import BeautifulSoup
import re
import sys
import glob

Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5105)
/home/anonoz/anaconda2/envs/tensorflow/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [3]:
X = [[0,0], [1,1]]
Y = [0, 1]
clf = svm.SVC()
clf.fit(X, Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [4]:
clf.predict([[0, 0.5]])

array([0])

## Step 1: Pre-processing functions & file loading

In [5]:
def tokenize_text(text):
    #1 Remove HTML (inspired by Kaggle)
    text = BeautifulSoup(text, "html.parser").getText()

    #2 Tokenize (stolen from Yoon Kim's CNN)
    text = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", text)     
    text = re.sub(r"\'s", " \'s", text) 
    text = re.sub(r"\'ve", " \'ve", text) 
    text = re.sub(r"n\'t", " n\'t", text) 
    text = re.sub(r"\'re", " \'re", text) 
    text = re.sub(r"\'d", " \'d", text) 
    text = re.sub(r"\'ll", " \'ll", text) 
    text = re.sub(r",", " , ", text) 
    text = re.sub(r"!", " ! ", text) 
    text = re.sub(r"\(", " \( ", text) 
    text = re.sub(r"\)", " \) ", text) 
    text = re.sub(r"\?", " \? ", text) 
    text = re.sub(r"\s{2,}", " ", text)
    
    # Step 3: 
    return text.lower().split()

## Step 2: Let Doc2Vec infer the vectors

In [6]:
d2v_model = Doc2Vec.load(doc2vec_model_file)

In [7]:
X_train = []
Y_train = []
X_test  = []
Y_test  = []
log_counter = 0

for txt in glob.glob(positive_review_txts):
    with (open(txt, 'r')) as f:
        review_vector = d2v_model.infer_vector(tokenize_text(f.read()))
        X_train.append(review_vector)
        Y_train.append(positive_y)
        log_counter += 1
        if log_counter % 100 == 0:
            sys.stdout.write("Training data: \r{0}".format(log_counter))
            sys.stdout.flush()
            
for txt in glob.glob(negative_review_txts):
    with (open(txt, 'r')) as f:
        review_vector = d2v_model.infer_vector(tokenize_text(f.read()))
        X_train.append(review_vector)
        Y_train.append(negative_y)
        log_counter += 1
        if log_counter % 100 == 0:
            sys.stdout.write("Training data: \r{0}".format(log_counter))
            sys.stdout.flush()
            
log_counter = 0
print("\n")
for txt in glob.glob(positive_review_vals):
    with (open(txt, 'r')) as f:
        review_vector = d2v_model.infer_vector(tokenize_text(f.read()))
        X_test.append(review_vector)
        Y_test.append(positive_y)
        log_counter += 1
        if log_counter % 100 == 0:
            sys.stdout.write("Testing data: \r{0}".format(log_counter))
            sys.stdout.flush()
            
for txt in glob.glob(negative_review_vals):
    with (open(txt, 'r')) as f:
        review_vector = d2v_model.infer_vector(tokenize_text(f.read()))
        X_test.append(review_vector)
        Y_test.append(negative_y)
        log_counter += 1
        if log_counter % 100 == 0:
            sys.stdout.write("Testing data: \r{0}".format(log_counter))
            sys.stdout.flush()

25000

25000

## Step 3: Fit SVM

In [23]:
clf = svm.LinearSVC()
clf.fit(X_train, Y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

## Test #1: SVM

In [25]:
clf.score(X_test, Y_test)

0.81091999999999997

## Test #2: Reviews

In [17]:
def test_predict(text):
    review_vector = d2v_model.infer_vector(tokenize_text(text))
    return clf.predict([review_vector])

In [18]:
test_predict('this movie is good')

array([1])

In [19]:
test_predict('fucking horrible')

array([0])

In [20]:
# RT bad - Inferno
test_predict('''

I kept hoping a Wayans brother would pop in and signal it was all one big genre parody, and when you're fervently wishing for a Wayans, you know you're in trouble.''')

array([0])

In [21]:
# RT positive Inferno
test_predict('''I've felt fondly towards Tom Hanks' Professor Robert Langdon ever since he paused, wild-eyed, while fleeing for his life in The Da Vinci Code, to say to his equally distraught companion: "I've got to get to a library. Fast!"''')

array([1])

In [22]:
# RT positive Doctor Strange
test_predict('''
An engaging, smartly cast and sporadically eye-popping addition to the studio's bulging portfolio.''')

array([0])

In [15]:
# RT positive Inception
test_predict('''
A spectacular fantasy thriller based on Nolan's own original screenplay, Inception is the smartest CGI head-trip since The Matrix.''')

array([1])

In [26]:
# IMDb positive - Saving Pvt Ryan
test_predict('''
I know it's fashionable to trash successful movies but at least be honest about the trashing... Pvt. Ryan was fiction but it was pretty good HISTORICAL fiction. The details were well thought out and based on reality.

There was nothing stupid about the portrayal of the German army... Rommel DID blunder in his placement of force, The high command DID think Calais was going to be the invasion spot, not Normandy. Hitler didn't wake up until noon on that day and his aides were afraid to wake him. The Rangers did come in right behind the first wave and did take a beach exit by sheer will to get the hell off the beach. The bluffs were the scene of heavy close fighting. The german defenders were mostly Eastern European conscripts from defeated areas. (note that the 2 men that tried to surrender were NOT speaking German). There WAS a young man rescued from interior Normandy after his brothers were all killed. He WAS an airborne trooper (the difference was that he was found by a chaplain and was removed from the front.)

The battles inside Normandy were small actions town to town, street to street, house to house. Small actions like taking the radar station happened. Small actions like a handful of men defending a river bridge against odds happened. Small squads of men, formed out of the misdrops banded together ad hoc to fight. There were all enlisted groups and all officer groups. A General did die in the glider assault. FUBAR aptly described much of what happened that day.

And there were only Americans in the movie because the Brits and Canadians were many klicks away in a different area... this was Omaha beach. The story was an American one. And Monty DID bog down the advance and everyone knew it. And as for "American Stereotypes"... well those pretty much define America: my college roomie was a wise-ass New York Jew. My best friend was a second generation east coast Sicilian. My college girlfriend was a third generation German. My first wife was French and English. I'm Irish, my boss is Norwegian and I work with a Navaho... you get the point?

So much for it being bad history. It was in fact an excellent way to let a jaded and somewhat ignorant-of-their-past generation *feel* something of what their grandparents (LIVING grandparents) went through. It is perhaps less important that the details be exact as the feel be right. Even now the details are not fully known or knowable about that campaign... it was too big, too complex and too chaotic to be knowable. There is not even an accurate casualty count of D-Day itself.

Now as to the depth of characters. What I saw there was the extraordinary circumstances into which ordinary people were thrown and what happened to them. I saw the things that would mark a generation (I have heard in my elderly male patients sentiments similar to what Cpt. Miller was expressing when he announced his ordinariness) I saw the dehumanization that occurs with war and its mitigation moment to moment, man to man... Cpt. Miller didn't know anything about Ryan and he didn't care... until Ryan revealed his humanity to him with his story of his brothers. Pvt. Reiban was ready to walk out of the situation until he discoverd his captains ordinariness and his humanity. Then he began to look to him almost as a father. Pvt. Mellish rightfully delights in his revenge for all the times he's had to take it because he was Jewish by telling German captives he's "Juden!" Nerdish Cpl. Upham can stand alongside his bigger, stronger, braver Ranger compatriots and describe the poetry and melancholy of Edith Piaf's song... then face his cowardice, turn around and stand up in the face of danger and finally demonstrate the dehumanization of the enterprise he was enmeshed in by executing Steamboat Willie... even though Willie had no more choice about being there than Upham did and in other circumstances would have made a friend.

I could go on and on with this but enough already. OK, perhaps it is not The Best Movie Ever Made but it is still a good movie. And if one will take the blinders of fashionable negativism off they will see it. Finally, this is not a patriotic story... if anything it is an acknowledgement and thank you to all those old men still out there that did so much for us. To them I say a deep and sincere thank you.''')

array([0])

In [27]:
# IMDb positive - Saving Pvt Ryan
test_predict('''
To think that this movie did not win Best Picture is a crime. Director Steven Spielberg uses all of his talent and resources to give to the world the greatest war film ever made.

Though it's true that this is not the type of movie you want to sit down with the family and eat popcorn, the emotional drive of the picture, the story's poignant messages, and the fantastic acting of the cast draws you into a world that is both dangerous and unpredictable.

Spielberg is able to take you into action and make you feel as if you are a participant in the movie and not just a viewer. This is Tom Hanks' best movie he ever did. Forget his performances in Philadelphia and Forrest Gump (though they were also good); he should have received another Oscar for the role of Capt. John Miller, a leader who must act strong in front of his men, but must also hide his emotions from them. It would have been well-deserved if he won again.

I give this movie my highest recommendation. Saving Private Ryan is a movie that makes you realize how life is precious and how honor and duty, though they are deep philosophical concepts that are praised in war, can put you in jeopardy of losing your life for something you may not believe in.

''')

array([1])

In [28]:
# IMDb 2/10 - Suicide Squad
test_predict('''
I don't get the ratings here. This is a cut and dry poorly made movie and fans of the DC universe deserve better. I don't normally post my reviews here. But I have to share my take on this movie because it just wasn't good. I didn't even have to go into spoilers to show how terrible it is. Movie goers shouldn't mindlessly consume these films. Christopher Nolan set a high bar, but producers and studios need to step messing with auteurs and maybe we can get a quality DC movie:

There is nothing in Suicide Squad that shows any hope that an auteur filmmaker can do anything distinctive with the current cash cow of the Hollywood machine: the super hero movie. What Christopher Nolan once made his own has devolved into a predictable pastiche whose charms should be wearing thin on audiences. It doesn't help that the movie is also an example of how bad one of these films can be when it becomes watered down and designed to refrain from shaking up anything in the so-called DC Universe. Suicide Squad, a PG-13 film, was supposed to be DC's entry to rival Marvel's R-rated Deadpool. Even though Deadpool had its own problems as a self-aware action movie, it still had focus and a bravado that is nowhere to be found in Suicide Squad.

Suicide Squad follows a group of villains with super powers released from prison as part of a government plan to protect the world from terrorists or whatever sign-of-the-times fear currently plaguing society (Zika?). Starring Will Smith as the hit man Deadshot and Margot Robbie as the Joker's manic girlfriend Harley Quinn, alongside several other less familiar DC baddies, these guys are supposed to be complex people who have long fallen from grace and are supposed to rise above to find their humanity and gain the audience's sympathy. But writer-director David Ayer tries so hard to take a safe route, you can see the gears trying to manipulate audience emotion, revealing the inherit problems of these comic book adaptations straining to catch up with decades of printed storytelling.

You can't totally blame Ayer, who last gave moviegoers Fury, an incredibly strong and startling war movie featuring a better fleshed out motley crew of characters. The preciousness Hollywood has for its ongoing world building of interconnected comic book films creates such tight restrictions on storytelling that anything that might upset that world has no room to prosper. At one point, toward the end of Suicide Squad, one character asks another, "Shouldn't you be dead?" Of course not, this is the DC universe, and it's gotta be milked. That means no major players should be written off in one movie.

The result of these storytelling restraints is a soulless kind of filmmaking hampered by pussyfooting. It's like a syrupy glaze that drowns out any possibility to shine above what has become a predictable pattern of storytelling. Characters dole out uninspired lines that play superficially to feelings, like, "Dad, I know you do bad things, but I still love you." Then there are the clichés, like "fight fire with fire." Sometimes the script inadvertently deflates the tension by spelling things out. Someone over a radio says, "Use extreme caution," and someone in the action responds, "I don't like this." But in case you miss that, someone else says, "I don't like it either." A kid playing with his action figures can come up with better chatter to establish tension....''')

array([0])

In [29]:
# IMDb 7/10 - Suicide Squad
test_predict('''To start it off, i'm a big fan of dc cinematic universe, and the year 2016, hadn't been a good year for me and the other fans. Even though, i'll state myself optimistic, since actually, all my hopes and expectation fall into this movie instead. 

OK, Let's start with the strong aspects of the movie; performances and characters. well, i actually think this is what the director want to top off and show to the audience, thus forgetting the plot, which we will get into later. but the characters and the performances are well-performed- Margot Robbie as Harley Quinn really tops off my expectation. i'm a fan of harley quinn character, and she actually brought me the harley quinn i'm expected for- sexy, beautiful, it girl of joker's. while watching the movie, and whenever i'm feeling bored, harley quinn is there to make the scene fresh. and i think all audience, including audiences who didn't know her will love her and feel sympathetic to her. Will smith as deadshot actually didn't deserve much of my appreciation. i mean even though he's one of the great and strong aspect in the movie, i just thought i don't see will smith even trying to act, which weirdly works. watching Will smith here is just watching him using a kickass suit. Jared Leto as The Joker, is actually isn't the worst part of the movie (unlike what people might have said) i think that Jared Leto pull of a great effort into his character, and some scene of him showing up gave me goosebumps and own joys. but, maybe how little the scene for him is, or his character is pretty under-developed that just doesn't fit the puzzle in the movie. Viola Davis as Amanda Waller also brought up my her characther pretty well, as she played the cold-hearted leader Amanda Waller. other performances are great, like Joel Kinnaman, Jai Courtney, Jay Hernandez, Karen Fukuhara, and Cara Delevigne.

The chemistry between the characters are great, as well as the development between their relationship. in my opinion, one of the strongest chemistry is Harley Quinn and Deadshot. Their chemistry is electric, funny, and watching them teasing, and slowly started to care about each other, is entertaining, and make me ship them together. even, i have a relationship that had a weak chemistry, that is Rick Flagg and June Moore's relationship which is bland, forced, and unnecessary. 

Now, parts that i think is the weak part of the movie, I think the plot is the main reason. The plot is lazy-written and under-developed,and at some point is pretty ridiculous. i don't want to get into it too much, since it will spoil some things, and i need you to watch the movie without spoilers, yet. even, i could tell you that the CGI is pretty cringy and sometimes, so does the visual effect between the scenes. the ending is actually rushed, predictable, and i think they could do better after building up the story of the fight scene.

At the end, everything has its bad and has its good, what i really suggest for you to do is just lower your expectation, yet don't miss it either. it may not lift the hype as in the greatness in the trailer, but i think it's still a great fun must see summer movie , and you should enjoy it without all the overly negative critics.''')

array([1])